In [0]:
spark.conf.set(
  "fs.azure.account.key.snowflakehexa.blob.core.windows.net",
  "s+dpBFiy973QoHY+hvsWQHQOtSglbQMOLcSPJzr6TT35DnKIzpdHgDuXHg7Ss9maUparnjBtbxrt+AStvpvY3A=="
)


In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType


In [0]:
schema = StructType([
    StructField("event_id", StringType(), True),
    StructField("event_time", StringType(), True),
    StructField("service_name", StringType(), True),
    StructField("log_level", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("api_endpoint", StringType(), True),
    StructField("response_time_ms", IntegerType(), True),
    StructField("error_message", StringType(), True),
    StructField("region", StringType(), True),
    StructField("_corrupt_record", StringType(), True)
])


In [0]:
df_raw = spark.read \
    .schema(schema) \
    .option("multiline", "true") \
    .json("wasbs://case@snowflakehexa.blob.core.windows.net/app_logs.json")



In [0]:
df_raw.count()
df_raw.show(truncate=False)



+--------+--------------------+-------------+---------+-------+--------------+----------------+-------------+------+---------------+
|event_id|event_time          |service_name |log_level|user_id|api_endpoint  |response_time_ms|error_message|region|_corrupt_record|
+--------+--------------------+-------------+---------+-------+--------------+----------------+-------------+------+---------------+
|evt_101 |2025-09-10T10:15:30Z|order-service|ERROR    |U1001  |/api/v1/orders|850             |Timeout      |IN    |NULL           |
+--------+--------------------+-------------+---------+-------+--------------+----------------+-------------+------+---------------+



In [0]:
from pyspark.sql.functions import col, to_timestamp, when

df_final = df_raw.select(
    col("event_id").alias("EVENT_ID"),
    to_timestamp("event_time").alias("EVENT_TIME"),
    col("service_name").alias("SERVICE_NAME"),
    col("log_level").alias("LOG_LEVEL"),
    col("user_id").alias("USER_ID"),
    col("api_endpoint").alias("API_ENDPOINT"),
    col("response_time_ms").alias("RESPONSE_TIME_MS"),
    col("error_message").alias("ERROR_MESSAGE"),
    col("region").alias("REGION"),
    when(col("log_level") == "ERROR", 1).otherwise(0).alias("ERROR_FLAG")
)


In [0]:
df_raw.write.mode("overwrite").format("delta").save("/tmp/app_logs_delta")


In [0]:
SNOWFLAKE_OPTIONS = {
    "sfURL": "GFCCSWR-TW94786.snowflakecomputing.com",
    "sfUser": "RAMKISHORESN",
    "sfPassword": "Ramkishoresn@06",
    "sfDatabase": "LOG_ANALYTICS_DB",
    "sfSchema": "PUBLIC",
    "sfWarehouse": "LOGS_WH"
}


In [0]:

df_final.write \
    .format("snowflake") \
    .options(**SNOWFLAKE_OPTIONS) \
    .option("dbtable", "APPLICATION_LOGS") \
    .mode("append") \
    .save()
